# **투표 대상 요청 관리**
- input : 대상 키워드 naver url, category_id
- DB 추가 전 확인 필수

### 1. 키워드 링크 입력

In [ ]:
import time
import re
import sys
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Chrome 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

def get_actor_profiles_from_links(link_list):
    """ 사용자가 직접 입력한 링크와 카테고리를 기반으로 프로필 정보를 크롤링하는 함수 """
    profile_list = []

    for line in link_list:
        try:
            profile_url, category = map(str.strip, line.split(",", 1))  # 링크와 카테고리 분리
        except ValueError:
            print(f"⚠️ 잘못된 형식: {line}")  # 형식이 올바르지 않을 경우 무시
            continue

        # URL에서 os 값 추출
        os_match = re.search(r"os=(\d+)", profile_url)
        os_value = os_match.group(1) if os_match else None
        if not os_value:
            print(f"⚠️ os 값을 찾을 수 없음: {profile_url}")
            continue

        driver.get(profile_url)
        time.sleep(1)  # 페이지 로딩 대기
        soup = BeautifulSoup(driver.page_source, "html.parser")  # Selenium 결과를 파싱

        profile_data = {"os": os_value, "category": category, "profile_url": profile_url}

        # 네이버 이름 크롤링
        naver_name_tag = soup.select_one("span.area_text_title strong._text")
        profile_data["naver_name"] = naver_name_tag.text.strip() if naver_name_tag else None

        # 기본 정보 크롤링
        for group in soup.select("div.detail_info div.info_group"):
            label_tag = group.select_one("dt")
            value_tag = group.select_one("dd")
            if label_tag and value_tag:
                label, value = label_tag.text.strip(), value_tag.text.strip()
                profile_data[label] = value

        # 프로필 이미지 크롤링
        image_tag = (
            soup.select_one("a.thumb._item img._img") or
            soup.select_one("div.img_scroll ul.img_list li._item:first-child img") or
            soup.select_one("a.thumb img._img")
        )
        profile_data["naver_image"] = image_tag["src"] if image_tag else None

        # 사이트 정보 크롤링
        site_dt = soup.select_one("dt:-soup-contains('사이트')")
        if site_dt:
            site_links = site_dt.find_next_sibling("dd").select("a")
            for link in site_links:
                profile_data[link.text.strip()] = link["href"]

        profile_list.append(profile_data)

    return pd.DataFrame(profile_list)

def clean_profile_data(df):
    """ 크롤링한 배우 프로필 데이터를 정리하는 함수 """
    df["os_source"] = "NAVER"

    # 출생일 변환 함수 (출생 컬럼이 있을 경우만 실행)
    if "출생" in df.columns:
        def format_birthdate(birth_text):
            match = re.search(r"(\d{4})\.(\d{2})\.(\d{2})", str(birth_text))
            return f"{match.group(1)}-{match.group(2)}-{match.group(3)}" if match else None

        df["출생"] = df["출생"].apply(format_birthdate)

    df["keyword"] = df["naver_name"]

    # 결측값을 빈 문자열("")로 변환 후 strip 적용
    df["keyword"] = df["keyword"].fillna("").astype(str).str.strip()
    df["naver_name"] = df["naver_name"].fillna("").astype(str).str.strip()

    # "다른이름" 컬럼이 없는 경우 기본값 추가
    if "다른이름" not in df.columns:
        df["다른이름"] = ""

    df["다른이름"] = df["다른이름"].fillna("").astype(str).str.strip()

    # combined_keyword 생성 (빈값 제외 후 중복 제거)
    df["combined_keyword"] = df.apply(lambda row: ", ".join(
        sorted(set(sum([x.split(", ") for x in [row["keyword"], row["naver_name"], row["다른이름"]] if x], [])))
    ), axis=1)

    # naver_info 생성 (빈 값 제외)
    df["naver_info"] = df.apply(lambda row: str({
        col: row[col] for col in df.columns
        if col not in ["os", "category", "profile_url", "keyword", "naver_name", "naver_image", "os_source", "combined_keyword"]
        and pd.notna(row[col]) and str(row[col]).strip()
    }), axis=1)

    # 컬럼 정리
    return df[["category", "os", "os_source", "keyword", "naver_name", "combined_keyword", "naver_info", "naver_image"]]

# 🔹 사용자 입력 받기
print("📌 크롤링할 링크와 카테고리를 입력하세요. (형식: 링크, 카테고리)")
print("   여러 개를 입력하려면 한 줄씩 입력하세요. (입력 종료: 빈 줄 입력 후 엔터)")

link_list = []
while True:
    user_input = input().strip()
    if not user_input:  # 빈 줄 입력 시 종료
        break
    link_list.append(user_input)

if link_list:
    df_profiles = get_actor_profiles_from_links(link_list)
    df_cleaned = clean_profile_data(df_profiles)

else:
    print("❌ 입력이 없습니다. 프로그램을 종료합니다.")

df_cleaned

📌 크롤링할 링크와 카테고리를 입력하세요. (형식: 링크, 카테고리)
   여러 개를 입력하려면 한 줄씩 입력하세요. (입력 종료: 빈 줄 입력 후 엔터)


,category,os,os_source,keyword,naver_name,combined_keyword,naver_info,naver_image
0,52,100185,NAVER,이연희,이연희,"Yeonhee Lee, 이연희","{'다른이름': 'Yeonhee Lee', '소속사': '사람엔터테인먼트', '데뷔...",https://search.pstatic.net/common?type=b&size=...
1,10,24830865,NAVER,이병찬,이병찬,이병찬,"{'사이트': '인스타그램, 유튜브, X(트위터)', '인스타그램': 'https:...",https://search.pstatic.net/common?type=b&size=...


### 2. DB 추가

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
import urllib.parse

# 🔹 MySQL 연결 정보
db_config = {
    "dialect": "mysql",
    "driver": "pymysql",
    "username": "username",
    "password": urllib.parse.quote("password"),  # '@' 처리
    "server": "server",
    "port": 3306,
    "database": "database",
}

# 🔹 SQLAlchemy 연결 URL 생성
DB_URL = "{dialect}+{driver}://{username}:{password}@{server}:{port}/{database}".format(**db_config)

# 🔹 DB 연결
engine = create_engine(DB_URL)

# 🔹 SQL 쿼리
insert_keyword_query = text("""
insert_keyword_query
""")

insert_middle_query = text("""
insert_middle_query
""")

# 🔹 기존 데이터에서 찾는 쿼리
select_existing_keyword_query = text("""
select_existing_keyword_query
""")

# 🔹 여러 개의 데이터를 한 번에 삽입
with engine.begin() as conn:
    for row in df_cleaned.to_dict(orient="records"):

        # 🔹 category_id 설정
        category_id = row.get("category", None)

        # 🔹 기존 데이터 확인
        keyword_id = None
        if # os 값이 존재하는 경우
            keyword_id = existing_keyword_id  # ✅ 기존 데이터 사용

        if keyword_id is None:  # 기존 데이터가 없으면 새로 삽입
            result = conn.execute(insert_keyword_query, row)
            keyword_id = conn.execute(text("SELECT LAST_INSERT_ID();")).scalar()

        # 🔹 keyword_id가 존재하는 경우
        if keyword_id is not None:
            conn.execute(insert_middle_query, {"keyword_id": keyword_id, "category_id": category_id})

print("데이터 삽입 완료!")


데이터 삽입 완료!
